## Imports


In [1]:
%load_ext autoreload
%autoreload 2

#Imports
import geopandas as gpd
import numpy as np
import pandas as pd

#k-means
import data_weighted_kmeans
import importlib

#graph
import folium
import random

#widget
import ipywidgets as widgets
import os
import time
import selenium
from selenium import webdriver


## Manage Data 

In [8]:
# Load data
iris = gpd.read_file("data/CONTOURS-IRIS_2-1__SHP__FRA_2017-06-30/CONTOURS-IRIS/1_DONNEES_LIVRAISON_2016/CONTOURS-IRIS_2-1_SHP_LAMB93_FE-2016/CONTOURS-IRIS.shp")

# Change projection
iris = iris.to_crs(epsg='4326')
iris.head()

population_2014 = pd.read_excel("data/population_iris/base-ic-evol-struct-pop-2014.xls", skiprows=4, header=1)


In [123]:
N=population_2014["P14_POP"].sum()
n=population_2014.groupby("DEP")["P14_POP"].sum()
N
h = 335*n/N
p=h.astype('int')

In [160]:
#distribute circonscription by departement
df_rep = pd.DataFrame()
df_rep['nb_circo'] = h.values
df_rep['nb_circo_int'] = p.values
df_rep['nb_circo_int'] = df_rep['nb_circo_int'].replace(0,1)
df_rep['dep']=population_2014.groupby("DEP")["DEP"]
rest = int(df_rep['nb_circo'].sum()-df_rep['nb_circo_int'].sum())+1
df_rep['nb_circo_reste'] = df_rep['nb_circo'] - df_rep['nb_circo_int']
df_rep = df_rep.sort_values('nb_circo_reste', ascending=True)
df_rep['nb_circo_reste_arrondi'] = (df_rep['nb_circo_reste'] >= df_rep['nb_circo_reste'].nlargest(n=rest).min()).astype('int') 
df_rep['circo_total'] = df_rep['nb_circo_int'] + df_rep['nb_circo_reste_arrondi'] 
df_rep = df_rep.sort_values('dep')
df_rep.head()

,nb_circo,nb_circo_int,dep,nb_circo_reste,nb_circo_reste_arrondi,circo_total
0,3.182546,3,"(01, [01, 01, 01, 01, 01, 01, 01, 01, 01, 01, ...",0.182546,0,3
1,2.743667,2,"(02, [02, 02, 02, 02, 02, 02, 02, 02, 02, 02, ...",0.743667,1,3
2,1.743752,1,"(03, [03, 03, 03, 03, 03, 03, 03, 03, 03, 03, ...",0.743752,1,2
3,0.821337,1,"(04, [04, 04, 04, 04, 04, 04, 04, 04, 04, 04, ...",-0.178663,0,1
4,0.711012,1,"(05, [05, 05, 05, 05, 05, 05, 05, 05, 05, 05, ...",-0.288988,0,1


In [161]:
iris_filtered= {}

for i in range(1,95):
    if i<10 :
        departement = "0"+str(i)
        
    else :
        departement = str(i)
    if i!=20:    
        iris_filtered[i] = iris[iris["CODE_IRIS"].str.startswith(departement)].copy()
        
    
population_2014["CODE_IRIS"] = population_2014["IRIS"]


dept = 1

for key, value in iris_filtered.items():
         
        value.loc[:,'P14_POP'] = population_2014['P14_POP']
        value.loc[:, 'departement_iris'] = dept
        dept = dept + 1
        value.loc[:, 'centroid_lng'] = value["geometry"].centroid.apply(lambda x: x.x)
        value.loc[:, 'centroid_lat'] = value["geometry"].centroid.apply(lambda x: x.y)    

## Create & save map

In [162]:
%%time
importlib.reload(data_weighted_kmeans)
mapa = folium.Map([46.575859, 0.290518],
              zoom_start=9,
              tiles='cartodbpositron')
map_filtered={}
k=335
count=1;
#for k in range(300,600,25):
for key, value in iris_filtered.items():
    if key != 59:
        points = []
        nb= df_rep['circo_total'][key]
        for idx, row in value.iterrows():
            points.append({"coords": np.array([float(row['centroid_lng']), float(row['centroid_lat'])]), \
                          "w": int(row["P14_POP"]), "zip": row['CODE_IRIS'], "state": row["departement_iris"]})

        centers = data_weighted_kmeans.randomize_initial_cluster(points, nb)

        points, centers, it_num = data_weighted_kmeans.kmeans_evolution_weighted(points, centers, nb, it_max=1000, weight_step_scale=10)
        points_df = pd.DataFrame.from_dict(points)
        points_df["CODE_IRIS"] = points_df["zip"]
        points_df["coords"] = "aaa"
        result = value.merge(points_df, how='inner', on=['CODE_IRIS', 'CODE_IRIS'])
        simplified_map = result.dissolve(by='c')
        simplified_map["colour"] = ["#%06x" % random.randint(0, 0xFFFFFF) for i in range(0,nb)]
        points = folium.features.GeoJson(simplified_map,  style_function=lambda feature: { 'fillColor': feature['properties']['colour'],
            'color' : "#000000",
            'weight' : 1,
            'fillOpacity' : 0.5,
            }
        )
        map_filtered["map{0}".format(key)] = points
        mapa.add_children(points)
        for i in range(0,len(centers)):
            center = centers[i ]
            folium.Marker([center["coords"][1], center["coords"][0]], popup="population : {pop}".format(idx=i, pop=center["pop"])).add_to(mapa)

fn='generated_map\map_'+str(k)+'circo.html'
mapa.save(fn)
    

C:\Users\LHOUMEAU\Anaconda3_\lib\site-packages\ipykernel_launcher.py:32: FutureWarning: Method `add_children` is deprecated. Please use `add_child` instead.


Wall time: 3min 38s
